# 🏥 Analyzing Medical Images with Google MedGemma 1.5
---

## 🎯 Abstract
Healthcare systems worldwide face escalating clinician burnout, widening health literacy gaps, and limited access to specialist care in resource-constrained settings. Physicians spend over half their time on documentation, patients often struggle to understand medical guidance, and rural clinics lack advanced diagnostic support.

This notebook demonstrates **Google's MedGemma 1.5** model for analyzing medical images including X-rays, CT scans, and ultrasounds.

### Goals:
- 🔍 Analyze chest X-rays and identify any abnormalities
- 🩺 Detect pulmonary conditions such as pneumonia, edema, effusions, asthma, etc
- 📊 Process ultrasound imaging
- 💡 Provide clinical contextually aware assessments

### ⚠️  Disclaimer
This notebook is for educational purposes only. Outputs should NOT replace professional medical diagnosis.

---

## 📦 1. Loading Libraries & Setup

First we will install the required packages

In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers accelerate
!pip install pillow requests
!pip install numpy

## ✅ 1.1 Environment Verification

Checking PyTorch version, CUDA availability, and GPU device.

In [ ]:
import torch
print(torch.__version__)          # >= 2.6.0
print(torch.cuda.is_available())  # True
print(torch.cuda.get_device_name(0))  # NVIDIA

2.9.1+cu128
True
NVIDIA H100 80GB HBM3


## 1.2 Hugging Face Authentication

Generating and inputting token from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

In [ ]:
from huggingface_hub import login
login(token="yourtoken")  # Replace with your Hugging Face token

## 🚀 2. Loading MedGemma Model

We will now load the **google/medgemma-1.5-4b-it** model with BFloat16 precision for GPU acceleration

In [ ]:
from transformers import pipeline
from PIL import Image
import requests
import torch

pipe = pipeline(
    "image-text-to-text",
    model="google/medgemma-1.5-4b-it",
    dtype=torch.bfloat16,
    device="cuda",
)



## 🔬3. Medical Imaging Analysis


### Case 1: Standard Chest X-Ray

**Objective:** The model will analyze a standard chest X-ray and identify any abnormalities.

The following are the key structures that will be evaluated:
* Lung fields heart size
* mediastinum
* bony structures

In [6]:
# Image attribution: Stillwaterising, CC0, via Wikimedia Commons
image_url = "https://www.cvmg.com/wp-content/uploads/2014/10/chest-xray.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "Describe this X-ray, whats wrong here?"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Based on the provided chest X-ray, here's a description and assessment:

**Description:**

*   The image is a standard AP (anteroposterior) chest X-ray.
*   The lungs appear clear with normal expansion.
*   The heart size seems within normal limits.
*   The mediastinum (the space between the lungs containing the heart, great vessels, trachea, etc.) appears normal in width.
*   The bony structures of the rib cage are intact.
*   The diaphragm is visible and appears normal in position.

**Potential Issues/Observations:**

*   There is a small, nodular opacity in the right upper lobe of the lung. This could be a benign lesion (such as a granuloma or hamartoma) or an early sign of a more serious condition. Further investigation may be warranted depending on the clinical context.

**Disclaimer:**

I am an AI and cannot provide medical diagnoses. This is not a substitute for a professional medical evaluation by a qualified radiologist or physician. A proper diagnosis requires a complete clin

### Case 2: Suspected Pneumonia

**Patient:** 58-year-old male

**Clinical History:**
- 3-month progressive shortness of breath
- Non-productive cough
- 7 kg weight loss over 8 weeks
- Low-grade fevers and night sweats
- No response to multiple antibiotic courses

**Differential:** Pneumonia, tuberculosis, lung malignancy, interstitial lung disease

In [8]:
image_url = "https://radiologyassistant.nl/assets/chest-x-ray-lung-disease/a50e9a848bc956_5a-NHL.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "A 58-year-old male with a 3-month history of progressive shortness of breath and non-productive cough presents for evaluation. He reports persistent fatigue, a 7-kg (15-lb) unintentional weight loss over the past 8 weeks, and intermittent low-grade fevers. He denies significant chest pain but notes occasional night sweats and decreased exercise tolerance. There has been no significant response to multiple courses of antibiotics prescribed for presumed lower respiratory tract infection.Describe this X-ray, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Based on the chest X-ray, the following observations are made:

*   **Right Lung:** There appears to be consolidation, possibly opacification or haziness, in the right upper and lower lung fields. The consolidations are patchy.
*   **Left Lung:** The left lung appears relatively clear.
*   **Heart Size:** The heart size seems normal.
*   **Mediastinum:** The mediastinum is unremarkable.
*   **Bones:** The bony structures appear intact.

**Conclusion:**

The X-ray findings are concerning for **pneumonia**. The patchy opacities in the right lung suggest inflammation and fluid accumulation within the lung tissue. While the history of cough, shortness of breath, and fatigue is suggestive of pulmonary infection, the progressive nature of the symptoms, weight loss, night sweats, and lack of response to antibiotics raise concerns for a more complex underlying cause. The patient should be further investigated to determine the precise diagnosis, which could be a complicated pneumonia, lung absc

### Case 3: Acute Respiratory Distress

**Patient:** 62-year-old

**Clinical History:**
- Acute onset dyspnea (past few days)
- Pleuritic chest pain
- Recent prolonged immobility (long trip)
- Intermittent lightheadedness

**Risk Factors:** DVT/PE risk from immobility

**Differential:**
* Pulmonary embolism
* pleural effusion
* pneumothorax

In [12]:
image_url = "https://radiologyassistant.nl/assets/chest-x-ray-lung-disease/a50e9d58ded69a_8-Longinfarct.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text","text": "A 62-year-old patient presents with acute onset of worsening shortness of breath over the past few days. The dyspnea began suddenly and has progressively limited physical activity. The patient also reports pleuritic chest discomfort, especially on deep inspiration, and a dry cough. There is a recent history of prolonged immobility after a long trip and intermittent episodes of lightheadedness. No fever, chills, or significant productive cough are noted.Describe this X-ray, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


Based on the provided chest X-ray, here is a concise description and assessment:

**X-ray Description:**

The image shows a frontal chest X-ray with a side view on the right.

**Findings:**

*   **Lungs:** The lungs appear relatively clear bilaterally.
*   **Pleura:** There is a significant pleural effusion on the right side, obscuring the right hemidiaphragm.
*   **Heart/Mediastinum:** The heart size appears within normal limits. The mediastinal structures are unremarkable.
*   **Bones:** The bony structures of the rib cage are visible.

**Possible Explanation**

Based on the presence of a large right-sided pleural effusion, a potential cause of shortness of breath and pleuritic chest discomfort is present. A possible underlying cause of this is a **pneumonia**, **tumor**, or **heart failure**.

**Further Evaluation**

To confirm the diagnosis and determine the cause, additional imaging or clinical data may be needed. Consider performing a CT scan of the chest to better evaluate the e

### Case 4: Suspected Heart Failure

**Patient:** 68-year-old

**Clinical History:**
- Progressive dyspnea (several weeks)
- Orthopnea (SOB when lying flat)
- Nocturnal cough with breathlessness
- Lower limb swelling

**Classic HF Features:** ✅ Orthopnea ✅ PND ✅ Peripheral edema

**Expected Findings:** Cardiomegaly, pulmonary edema, bilateral infiltrates

In [14]:
image_url = "https://radiologyassistant.nl/assets/chest-x-ray-lung-disease/a509797a678f8e_CHF-1c1.jpg"
image = Image.open(requests.get(image_url, headers={"User-Agent": "example"}, stream=True).raw)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text","text": "A 68-year-old individual presents to the clinic with progressively worsening shortness of breath over the past several weeks. The dyspnea is present at rest and worsens with exertion. The patient also reports intermittent orthopnea (shortness of breath when lying flat) and nocturnal cough with brief episodes of waking up breathless at night. There is associated fatigue and decreased exercise tolerance. The patient denies chest pain, hemoptysis, or high fever. There has been a gradual increase in lower limb swelling over the last few weeks.Describe this X-ray, whats wrong here?please provide a consice output"}
        ]
    }
]

output = pipe(text=messages, max_new_tokens=2000)
print(output[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


This chest X-ray shows bilateral pulmonary infiltrates, with a predominantly perihilar distribution. The heart size appears enlarged.

The overall presentation is suggestive of pulmonary edema, a condition where there is an excess fluid buildup in the lungs. Possible causes of pulmonary edema include heart failure, fluid overload, kidney disease, and pneumonia.

Given the patient's symptoms of shortness of breath, orthopnea, nocturnal cough, and lower limb swelling, heart failure is a strong consideration.

It would be important to consider other potential causes, such as pneumonia or fluid overload, and to correlate these findings with the patient's clinical history and physical exam. Further evaluation, including cardiac function testing, blood work, and possibly a pulmonary function test, would be required to determine the underlying etiology.
Final Answer: The final answer is $\boxed{Pulmonary edema}$


# 5. 📊 Conclusion

## ✅ Key Takeaways

**MedGemma 1.5 Capabilities:**
- ✓ Multimodal understanding (image + text)
- ✓ Clinical context awareness
- ✓ Multiple imaging modalities
- ✓ Detailed anatomical descriptions

## ⚠️ Limitations
- **Not a diagnostic tool** - requires professional validation
- **Educational purpose only**
- **Clinical judgment essential**

## 📚 References
- [MedGemma Model](https://huggingface.co/google/medgemma-1.5-4b-it)
- [Radiology Assistant](https://radiologyassistant.nl/)